# Loan Prediction

1. Data Collection
2. Data Assessment
3. Data Cleaning
4. Data Pre-processing
5. model training
6. Model Optimization
7. model selection
8. save trained model for deployment

## 1. Data Collection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
df = pd.read_csv('lending_club_loans.csv')
df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_2876\1255256620.py:1: DtypeWarning: Columns (0,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lending_club_loans.csv')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


## 2. Data Assessment

In [3]:
df.shape

(42542, 115)

In [4]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=115)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42542 entries, 0 to 42541
Columns: 115 entries, id to total_il_high_credit_limit
dtypes: float64(90), object(25)
memory usage: 37.3+ MB


In [6]:
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,4.253500e+04,42535.000000,42535.000000,42535.000000,42535.000000,4.253100e+04,42535.000000,42506.000000,42535.000000,42535.000000,...,0.0,0.0,0.0,0.0,41170.000000,42430.000000,0.0,0.0,0.0,0.0
mean,8.257026e+05,11089.722581,10821.585753,10139.830603,322.623063,6.913656e+04,13.373043,0.152449,713.052545,717.052545,...,NaN,NaN,NaN,NaN,0.045227,0.000024,NaN,NaN,NaN,NaN
std,2.795409e+05,7410.938391,7146.914675,7131.686447,208.927216,6.409635e+04,6.726315,0.512406,36.188439,36.188439,...,NaN,NaN,NaN,NaN,0.208737,0.004855,NaN,NaN,NaN,NaN
min,7.047300e+04,500.000000,500.000000,0.000000,15.670000,1.896000e+03,0.000000,0.000000,610.000000,614.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
25%,6.384795e+05,5200.000000,5000.000000,4950.000000,165.520000,4.000000e+04,8.200000,0.000000,685.000000,689.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
50%,8.241780e+05,9700.000000,9600.000000,8500.000000,277.690000,5.900000e+04,13.470000,0.000000,710.000000,714.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
75%,1.033946e+06,15000.000000,15000.000000,14000.000000,428.180000,8.250000e+04,18.680000,0.000000,740.000000,744.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
max,1.314167e+06,35000.000000,35000.000000,35000.000000,1305.190000,6.000000e+06,29.990000,13.000000,825.000000,829.000000,...,NaN,NaN,NaN,NaN,2.000000,1.000000,NaN,NaN,NaN,NaN


In [7]:
df.isnull().sum()

id                                4
member_id                         7
loan_amnt                         7
funded_amnt                       7
funded_amnt_inv                   7
                              ...  
tax_liens                       112
tot_hi_cred_lim               42542
total_bal_ex_mort             42542
total_bc_limit                42542
total_il_high_credit_limit    42542
Length: 115, dtype: int64

In [8]:
df.duplicated().sum()

3

In [9]:
df['pub_rec_bankruptcies'].value_counts()

0.0    39316
1.0     1846
2.0        8
Name: pub_rec_bankruptcies, dtype: int64

In [10]:
# Set the display options to show all columns
pd.set_option('display.max_columns', None)

df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgrade...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.70%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,Sep-16,744.0,740.0,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use thi...,car,bike,309xx,GA,1.00,0.0,Apr-99,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.40%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-16,499.0,0.0,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,735.0,739.0,2.0,NaN,NaN,2.0,0.0,2956.0,98.50%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,Sep-16,719.0,715.0,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-96,690.0,694.0,1.0,35.0,NaN,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,

In [11]:
df.isnull().sum()

id                                4
member_id                         7
loan_amnt                         7
funded_amnt                       7
funded_amnt_inv                   7
                              ...  
tax_liens                       112
tot_hi_cred_lim               42542
total_bal_ex_mort             42542
total_bc_limit                42542
total_il_high_credit_limit    42542
Length: 115, dtype: int64

In [12]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=115)

In [13]:
# Find rows with at least one null value
rows_with_nulls = df[df.isnull().any(axis=1)]

# Print the rows with null values
rows_with_nulls

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgrade...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.70%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,Sep-16,744.0,740.0,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use thi...,car,bike,309xx,GA,1.00,0.0,Apr-99,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.40%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-16,499.0,0.0,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,735.0,739.0,2.0,NaN,NaN,2.0,0.0,2956.0,98.50%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,Sep-16,719.0,715.0,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-96,690.0,694.0,1.0,35.0,NaN,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,

In [14]:
# Count the number of null values in each column
null_counts = df.isnull().sum()

# Filter columns with null values
columns_with_nulls = null_counts[null_counts > 0]

# Get the names of columns with null values
columns_with_null_names = columns_with_nulls.index.tolist()

# Print columns with null values and their corresponding null counts
print("Columns with null values:")
for column_name in columns_with_null_names:
    print(f"{column_name}: {columns_with_nulls[column_name]} null values")

# Get all columns names (all 115 features)
all_columns = df.columns.tolist()

# Print all column names (115 features)
print("All column names:")
print(all_columns)


Columns with null values:
id: 4 null values
member_id: 7 null values
loan_amnt: 7 null values
funded_amnt: 7 null values
funded_amnt_inv: 7 null values
term: 7 null values
int_rate: 7 null values
installment: 7 null values
grade: 7 null values
sub_grade: 7 null values
emp_title: 2633 null values
emp_length: 1119 null values
home_ownership: 7 null values
annual_inc: 11 null values
verification_status: 7 null values
issue_d: 7 null values
loan_status: 7 null values
pymnt_plan: 7 null values
url: 7 null values
desc: 13526 null values
purpose: 7 null values
title: 20 null values
zip_code: 7 null values
addr_state: 7 null values
dti: 7 null values
delinq_2yrs: 36 null values
earliest_cr_line: 36 null values
fico_range_low: 7 null values
fico_range_high: 7 null values
inq_last_6mths: 36 null values
mths_since_last_delinq: 26933 null values
mths_since_last_record: 38891 null values
open_acc: 36 null values
pub_rec: 36 null values
revol_bal: 7 null values
revol_util: 97 null values
total_acc: 

In [15]:
# Count the number of null values in each column
null_counts = df.isnull().sum()

# Filter columns with null values
columns_with_nulls = null_counts[null_counts > 0]

# Print columns with null values and their corresponding null counts
columns_with_nulls


id                                4
member_id                         7
loan_amnt                         7
funded_amnt                       7
funded_amnt_inv                   7
                              ...  
tax_liens                       112
tot_hi_cred_lim               42542
total_bal_ex_mort             42542
total_bc_limit                42542
total_il_high_credit_limit    42542
Length: 115, dtype: int64

## 3. data cleaning
1. remove duplicate
2. drop features with null values

In [16]:
columns_to_drop = ['id', 'member_id', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit','total_il_high_credit_limit', 
                   'tax_liens','mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 
                   'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 
                   'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 
                   'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 
                   'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
                   'percent_bc_gt_75','annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 
                   'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 
                   'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 
                   'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 
                   'bc_util']

In [17]:
# Drop the specified columns
df = df.drop(columns=columns_to_drop)

# Print the DataFrame after dropping columns
df.head()


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgrade...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.70%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,Sep-16,744.0,740.0,0.0,NaN,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use thi...,car,bike,309xx,GA,1.00,0.0,Apr-99,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.40%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-16,499.0,0.0,0.0,NaN,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,735.0,739.0,2.0,NaN,NaN,2.0,0.0,2956.0,98.50%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,Sep-16,719.0,715.0,0.0,NaN,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-96,690.0,694.0,1.0,35.0,NaN,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,NaN,Apr-16,604.0,600.0,0.0,NaN,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-96,695.0,699.0,0.0,38.0,NaN,15.0,0.0,27783.0,53.90%,38.0,f,270.78,270.78,3784.490000,3784.49,2729.22,1055.27,0.00,0.00,0.00,Sep-16,67.79,Oct-16,Sep-16,694.0,690.0,0.0,NaN,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0


In [18]:
df.shape

(42542, 59)

In [19]:
df['acc_now_delinq'].value_counts()

0.0    42502
1.0        4
Name: acc_now_delinq, dtype: int64

In [20]:
columns_to_drop = ['url', 'desc','zip_code', 'addr_state', 'pymnt_plan', 'initial_list_status','out_prncp', 
                   'out_prncp_inv', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'acc_now_delinq', 
                   'chargeoff_within_12_mths','delinq_amnt', 'mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 
                   'emp_title', 'title']


# Drop the specified columns
df = df.drop(columns=columns_to_drop)

# Print the DataFrame after dropping columns
df.head()


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,policy_code,application_type,pub_rec_bankruptcies
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,credit_card,27.65,0.0,Jan-85,735.0,739.0,1.0,3.0,0.0,13648.0,83.70%,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,Sep-16,744.0,740.0,1.0,INDIVIDUAL,0.0
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,car,1.00,0.0,Apr-99,740.0,744.0,5.0,3.0,0.0,1687.0,9.40%,4.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-16,499.0,0.0,1.0,INDIVIDUAL,0.0
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,small_business,8.72,0.0,Nov-01,735.0,739.0,2.0,2.0,0.0,2956.0,98.50%,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,Sep-16,719.0,715.0,1.0,INDIVIDUAL,0.0
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,other,20.00,0.0,Feb-96,690.0,694.0,1.0,10.0,0.0,5598.0,21%,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,Apr-16,604.0,600.0,1.0,INDIVIDUAL,0.0
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,1 year,RENT,80000.0,Source Verified,Dec-11,Current,other,17.94,0.0,Jan-96,695.0,699.0,0.0,15.0,0.0,27783.0,53.90%,38.0,3784.490000,3784.49,2729.22,1055.27,0.00,0.00,0.00,Sep-16,67.79,Sep-16,694.0,690.0,1.0,INDIVIDUAL,0.0


In [21]:
df.shape

(42542, 41)

#### checking columns again for further investigation

In [22]:
# Count the number of null values in each column
null_counts = df.isnull().sum()

# Filter columns with null values
columns_with_nulls = null_counts[null_counts > 0]

# Get the names of columns with null values
columns_with_null_names = columns_with_nulls.index.tolist()

# Print columns with null values and their corresponding null counts
print("Columns with null values:")
for column_name in columns_with_null_names:
    print(f"{column_name}: {columns_with_nulls[column_name]} null values")

# Get all columns names (all 115 features)
all_columns = df.columns.tolist()

# Print all column names (115 features)
print("All column names:")
print(all_columns)


Columns with null values:
loan_amnt: 7 null values
funded_amnt: 7 null values
funded_amnt_inv: 7 null values
term: 7 null values
int_rate: 7 null values
installment: 7 null values
grade: 7 null values
sub_grade: 7 null values
emp_length: 1119 null values
home_ownership: 7 null values
annual_inc: 11 null values
verification_status: 7 null values
issue_d: 7 null values
loan_status: 7 null values
purpose: 7 null values
dti: 7 null values
delinq_2yrs: 36 null values
earliest_cr_line: 36 null values
fico_range_low: 7 null values
fico_range_high: 7 null values
inq_last_6mths: 36 null values
open_acc: 36 null values
pub_rec: 36 null values
revol_bal: 7 null values
revol_util: 97 null values
total_acc: 36 null values
total_pymnt: 7 null values
total_pymnt_inv: 7 null values
total_rec_prncp: 7 null values
total_rec_int: 7 null values
total_rec_late_fee: 7 null values
recoveries: 7 null values
collection_recovery_fee: 7 null values
last_pymnt_d: 90 null values
last_pymnt_amnt: 7 null values
last

In [23]:
df.isnull().sum()

loan_amnt                     7
funded_amnt                   7
funded_amnt_inv               7
term                          7
int_rate                      7
installment                   7
grade                         7
sub_grade                     7
emp_length                 1119
home_ownership                7
annual_inc                   11
verification_status           7
issue_d                       7
loan_status                   7
purpose                       7
dti                           7
delinq_2yrs                  36
earliest_cr_line             36
fico_range_low                7
fico_range_high               7
inq_last_6mths               36
open_acc                     36
pub_rec                      36
revol_bal                     7
revol_util                   97
total_acc                    36
total_pymnt                   7
total_pymnt_inv               7
total_rec_prncp               7
total_rec_int                 7
total_rec_late_fee            7
recoveri

In [24]:
# Drop rows with any null values
df = df.dropna()

In [25]:
df.shape

(39930, 41)

In [26]:
df.duplicated().sum()

0

In [27]:
# Create a copy of the DataFrame
df_cleaned = df.copy()

## 5. Data Pre-processing 

In [28]:
df_cleaned.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,policy_code,application_type,pub_rec_bankruptcies
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,credit_card,27.65,0.0,Jan-85,735.0,739.0,1.0,3.0,0.0,13648.0,83.70%,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,Sep-16,744.0,740.0,1.0,INDIVIDUAL,0.0
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,car,1.00,0.0,Apr-99,740.0,744.0,5.0,3.0,0.0,1687.0,9.40%,4.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-16,499.0,0.0,1.0,INDIVIDUAL,0.0
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,small_business,8.72,0.0,Nov-01,735.0,739.0,2.0,2.0,0.0,2956.0,98.50%,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,Sep-16,719.0,715.0,1.0,INDIVIDUAL,0.0
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,other,20.00,0.0,Feb-96,690.0,694.0,1.0,10.0,0.0,5598.0,21%,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,Apr-16,604.0,600.0,1.0,INDIVIDUAL,0.0
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,1 year,RENT,80000.0,Source Verified,Dec-11,Current,other,17.94,0.0,Jan-96,695.0,699.0,0.0,15.0,0.0,27783.0,53.90%,38.0,3784.490000,3784.49,2729.22,1055.27,0.00,0.00,0.00,Sep-16,67.79,Sep-16,694.0,690.0,1.0,INDIVIDUAL,0.0


In [29]:
df_cleaned['term'] = df_cleaned['term'].str.split().str[0].astype(int)
df_cleaned['int_rate'] = df_cleaned['int_rate'].astype(str).str.replace('%', '').astype(float)
df_cleaned['revol_util'] = df_cleaned['revol_util'].astype(str).str.replace('%', '').astype(float)

In [30]:
df_cleaned.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,policy_code,application_type,pub_rec_bankruptcies
0,5000.0,5000.0,4975.0,36,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,credit_card,27.65,0.0,Jan-85,735.0,739.0,1.0,3.0,0.0,13648.0,83.7,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,Sep-16,744.0,740.0,1.0,INDIVIDUAL,0.0
1,2500.0,2500.0,2500.0,60,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,car,1.00,0.0,Apr-99,740.0,744.0,5.0,3.0,0.0,1687.0,9.4,4.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-16,499.0,0.0,1.0,INDIVIDUAL,0.0
2,2400.0,2400.0,2400.0,36,15.96,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,small_business,8.72,0.0,Nov-01,735.0,739.0,2.0,2.0,0.0,2956.0,98.5,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,Sep-16,719.0,715.0,1.0,INDIVIDUAL,0.0
3,10000.0,10000.0,10000.0,36,13.49,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,other,20.00,0.0,Feb-96,690.0,694.0,1.0,10.0,0.0,5598.0,21.0,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,Apr-16,604.0,600.0,1.0,INDIVIDUAL,0.0
4,3000.0,3000.0,3000.0,60,12.69,67.79,B,B5,1 year,RENT,80000.0,Source Verified,Dec-11,Current,other,17.94,0.0,Jan-96,695.0,699.0,0.0,15.0,0.0,27783.0,53.9,38.0,3784.490000,3784.49,2729.22,1055.27,0.00,0.00,0.00,Sep-16,67.79,Sep-16,694.0,690.0,1.0,INDIVIDUAL,0.0


### Feature Encoding 

In [31]:
column_to_encode = ['grade', 'sub_grade','home_ownership', 'emp_length', 'verification_status', 'loan_status' , 'purpose', 'application_type']

In [32]:
from sklearn.preprocessing import LabelEncoder
# Columns to perform label encoding on
columns_to_encode = ['grade', 'sub_grade', 'home_ownership', 'emp_length', 'verification_status', 'loan_status', 'purpose', 'application_type']

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Apply label encoding to each column in columns_to_encode
for column in columns_to_encode:
    df_cleaned[column] = label_encoder.fit_transform(df_cleaned[column])

# Print the DataFrame after label encoding
df_cleaned.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,policy_code,application_type,pub_rec_bankruptcies
0,5000.0,5000.0,4975.0,36,10.65,162.87,1,6,1,3,24000.0,2,Dec-11,5,1,27.65,0.0,Jan-85,735.0,739.0,1.0,3.0,0.0,13648.0,83.7,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,Sep-16,744.0,740.0,1.0,0,0.0
1,2500.0,2500.0,2500.0,60,15.27,59.83,2,13,10,3,30000.0,1,Dec-11,0,0,1.00,0.0,Apr-99,740.0,744.0,5.0,3.0,0.0,1687.0,9.4,4.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-16,499.0,0.0,1.0,0,0.0
2,2400.0,2400.0,2400.0,36,15.96,84.33,2,14,1,3,12252.0,0,Dec-11,5,11,8.72,0.0,Nov-01,735.0,739.0,2.0,2.0,0.0,2956.0,98.5,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,Sep-16,719.0,715.0,1.0,0,0.0
3,10000.0,10000.0,10000.0,36,13.49,339.31,2,10,1,3,49200.0,1,Dec-11,5,9,20.00,0.0,Feb-96,690.0,694.0,1.0,10.0,0.0,5598.0,21.0,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,Apr-16,604.0,600.0,1.0,0,0.0
4,3000.0,3000.0,3000.0,60,12.69,67.79,1,9,0,3,80000.0,1,Dec-11,1,9,17.94,0.0,Jan-96,695.0,699.0,0.0,15.0,0.0,27783.0,53.9,38.0,3784.490000,3784.49,2729.22,1055.27,0.00,0.00,0.00,Sep-16,67.79,Sep-16,694.0,690.0,1.0,0,0.0


In [33]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39930 entries, 0 to 42476
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   loan_amnt                39930 non-null  float64
 1   funded_amnt              39930 non-null  float64
 2   funded_amnt_inv          39930 non-null  float64
 3   term                     39930 non-null  int32  
 4   int_rate                 39930 non-null  float64
 5   installment              39930 non-null  float64
 6   grade                    39930 non-null  int32  
 7   sub_grade                39930 non-null  int32  
 8   emp_length               39930 non-null  int32  
 9   home_ownership           39930 non-null  int32  
 10  annual_inc               39930 non-null  float64
 11  verification_status      39930 non-null  int32  
 12  issue_d                  39930 non-null  object 
 13  loan_status              39930 non-null  int32  
 14  purpose               

In [34]:
# Columns to fix
date_columns_to_fix = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

# Loop through each column and convert to standard datetime format
for column in date_columns_to_fix:
    df_cleaned[column] = pd.to_datetime(df_cleaned[column], format='%b-%y')
    

# Loop through each date column
for column in date_columns_to_fix:
    # Extract year and month from the date column
    df_cleaned[f'{column}_year'] = pd.to_datetime(df_cleaned[column]).dt.year
    df_cleaned[f'{column}_month'] = pd.to_datetime(df_cleaned[column]).dt.month

    # Convert year and month columns to int
    df_cleaned[f'{column}_year'] = df_cleaned[f'{column}_year'].astype(int)
    df_cleaned[f'{column}_month'] = df_cleaned[f'{column}_month'].astype(int)
    
     # Drop the original date column
    df_cleaned = df_cleaned.drop(column, axis=1)

In [35]:
df_cleaned.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,policy_code,application_type,pub_rec_bankruptcies,issue_d_year,issue_d_month,earliest_cr_line_year,earliest_cr_line_month,last_pymnt_d_year,last_pymnt_d_month,last_credit_pull_d_year,last_credit_pull_d_month
0,5000.0,5000.0,4975.0,36,10.65,162.87,1,6,1,3,24000.0,2,5,1,27.65,0.0,735.0,739.0,1.0,3.0,0.0,13648.0,83.7,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,171.62,744.0,740.0,1.0,0,0.0,2011,12,1985,1,2015,1,2016,9
1,2500.0,2500.0,2500.0,60,15.27,59.83,2,13,10,3,30000.0,1,0,0,1.00,0.0,740.0,744.0,5.0,3.0,0.0,1687.0,9.4,4.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,119.66,499.0,0.0,1.0,0,0.0,2011,12,1999,4,2013,4,2016,9
2,2400.0,2400.0,2400.0,36,15.96,84.33,2,14,1,3,12252.0,0,5,11,8.72,0.0,735.0,739.0,2.0,2.0,0.0,2956.0,98.5,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,649.91,719.0,715.0,1.0,0,0.0,2011,12,2001,11,2014,6,2016,9
3,10000.0,10000.0,10000.0,36,13.49,339.31,2,10,1,3,49200.0,1,5,9,20.00,0.0,690.0,694.0,1.0,10.0,0.0,5598.0,21.0,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,357.48,604.0,600.0,1.0,0,0.0,2011,12,1996,2,2015,1,2016,4
4,3000.0,3000.0,3000.0,60,12.69,67.79,1,9,0,3,80000.0,1,1,9,17.94,0.0,695.0,699.0,0.0,15.0,0.0,27783.0,53.9,38.0,3784.490000,3784.49,2729.22,1055.27,0.00,0.00,0.00,67.79,694.0,690.0,1.0,0,0.0,2011,12,1996,1,2016,9,2016,9


In [36]:
# Save the preprocessed data to a CSV file
df_cleaned.to_csv('preprocessed_data.csv', index=False)

print("Preprocessed data saved to 'preprocessed_data.csv'")


Preprocessed data saved to 'preprocessed_data.csv'


## 6. Model Training and Evalution

In [37]:
df_processed = pd.read_csv('preprocessed_data.csv')
df_processed.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,policy_code,application_type,pub_rec_bankruptcies,issue_d_year,issue_d_month,earliest_cr_line_year,earliest_cr_line_month,last_pymnt_d_year,last_pymnt_d_month,last_credit_pull_d_year,last_credit_pull_d_month
0,5000.0,5000.0,4975.0,36,10.65,162.87,1,6,1,3,24000.0,2,5,1,27.65,0.0,735.0,739.0,1.0,3.0,0.0,13648.0,83.7,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,171.62,744.0,740.0,1.0,0,0.0,2011,12,1985,1,2015,1,2016,9
1,2500.0,2500.0,2500.0,60,15.27,59.83,2,13,10,3,30000.0,1,0,0,1.00,0.0,740.0,744.0,5.0,3.0,0.0,1687.0,9.4,4.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,119.66,499.0,0.0,1.0,0,0.0,2011,12,1999,4,2013,4,2016,9
2,2400.0,2400.0,2400.0,36,15.96,84.33,2,14,1,3,12252.0,0,5,11,8.72,0.0,735.0,739.0,2.0,2.0,0.0,2956.0,98.5,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,649.91,719.0,715.0,1.0,0,0.0,2011,12,2001,11,2014,6,2016,9
3,10000.0,10000.0,10000.0,36,13.49,339.31,2,10,1,3,49200.0,1,5,9,20.00,0.0,690.0,694.0,1.0,10.0,0.0,5598.0,21.0,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,357.48,604.0,600.0,1.0,0,0.0,2011,12,1996,2,2015,1,2016,4
4,3000.0,3000.0,3000.0,60,12.69,67.79,1,9,0,3,80000.0,1,1,9,17.94,0.0,695.0,699.0,0.0,15.0,0.0,27783.0,53.9,38.0,3784.490000,3784.49,2729.22,1055.27,0.00,0.00,0.00,67.79,694.0,690.0,1.0,0,0.0,2011,12,1996,1,2016,9,2016,9


In [38]:
X = df_processed.drop(['loan_status'], axis = 1)
y = df_processed['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

In [39]:
def evaluate_model(true, predicted):
    accuracy = accuracy_score(true, predicted)
    precision, recall, f1, _ = precision_recall_fscore_support(true, predicted, average='weighted')
    return accuracy, precision, recall, f1

In [40]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "XGBClassifier": XGBClassifier(), 
    "CatBoosting Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier": AdaBoostClassifier()
}

model_list = []
accuracy_list = []

for model_name, model in models.items():
    model.fit(X_train, y_train)  # Train the model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_accuracy, model_train_precision, model_train_recall, model_train_f1 = evaluate_model(y_train, y_train_pred)
    model_test_accuracy, model_test_precision, model_test_recall, model_test_f1 = evaluate_model(y_test, y_test_pred)

    print(model_name)
    model_list.append(model_name)
    
    print('Model performance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print("- Precision: {:.4f}".format(model_train_precision))
    print("- Recall: {:.4f}".format(model_train_recall))
    print("- F1 Score: {:.4f}".format(model_train_f1))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Accuracy: {:.4f}".format(model_test_accuracy))
    print("- Precision: {:.4f}".format(model_test_precision))
    print("- Recall: {:.4f}".format(model_test_recall))
    print("- F1 Score: {:.4f}".format(model_test_f1))
    
    print('='*35)
    print('\n')


C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit

Logistic Regression
Model performance for Training set
- Accuracy: 0.9289
- Precision: 0.9133
- Recall: 0.9289
- F1 Score: 0.9069
----------------------------------
Model performance for Test set
- Accuracy: 0.9279
- Precision: 0.9184
- Recall: 0.9279
- F1 Score: 0.9044




C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Decision Tree
Model performance for Training set
- Accuracy: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- F1 Score: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.9801
- Precision: 0.9804
- Recall: 0.9801
- F1 Score: 0.9802




C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGBClassifier
Model performance for Training set
- Accuracy: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- F1 Score: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.9937
- Precision: 0.9932
- Recall: 0.9937
- F1 Score: 0.9935




C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CatBoosting Classifier
Model performance for Training set
- Accuracy: 0.9991
- Precision: 0.9991
- Recall: 0.9991
- F1 Score: 0.9991
----------------------------------
Model performance for Test set
- Accuracy: 0.9919
- Precision: 0.9913
- Recall: 0.9919
- F1 Score: 0.9915


AdaBoost Classifier
Model performance for Training set
- Accuracy: 0.9149
- Precision: 0.8880
- Recall: 0.9149
- F1 Score: 0.8984
----------------------------------
Model performance for Test set
- Accuracy: 0.9130
- Precision: 0.8874
- Recall: 0.9130
- F1 Score: 0.8968




C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Overfitting Problem Suspected
##### Fixing Overfitting Problem Suspected

## 7. Model Optimization
* Model Regularization on Randomforestclassifier
* Model Regularization on Xgbootclassifier
* Model Regularization on Support Vector Machine

In [45]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest classifier with default settings
rf_model = RandomForestClassifier()

# Set hyperparameters for regularization
params = {
    'max_depth': 5,       # Maximum depth of a tree
    'min_samples_split': 2,  # Minimum number of samples required to split an internal node
    'min_samples_leaf': 1,   # Minimum number of samples required to be at a leaf node
    'max_features': 'auto',  # Number of features to consider when looking for the best split
    'bootstrap': True       # Whether bootstrap samples are used when building trees
}

# Update the model with the regularization parameters
rf_model.set_params(**params)

# Train and evaluate the model using your training and test data
rf_model.fit(X_train, y_train)


# Train and evaluate the model using your training data
y_train_pred_rf = rf_model.predict(X_train)
# Calculate performance metrics for training data
model_train_accuracy_rf, model_train_precision_rf, model_train_recall_rf, model_train_f1_rf = evaluate_model(y_train, y_train_pred_rf)

# Evaluate the model's performance on the test set
y_pred_rf = rf_model.predict(X_test)
# Calculate performance metrics for test data
model_test_accuracy_rf, model_test_precision_rf, model_test_recall_rf, model_test_f1_rf = evaluate_model(y_test, y_pred_rf)

# Print performance metrics for training data
print("Regularized Random Forest Model performance on training set:") 
print(f"Accuracy: {model_train_accuracy_rf}")
print(f"Precision: {model_train_precision_rf}")
print(f"Recall: {model_train_recall_rf}")
print(f"F1 Score: {model_train_f1_rf}")
print("===================================")

# Print performance metrics for test data
print("Regularized Random Forest Model performance on test set:")
print(f"Accuracy: {model_test_accuracy_rf}")
print(f"Precision: {model_test_precision_rf}")
print(f"Recall: {model_test_recall_rf}")
print(f"F1 Score: {model_test_f1_rf}")


C:\Users\HP\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Regularized Random Forest Model performance on training set:
Accuracy: 0.9238041572752317
Precision: 0.9136360699370714
Recall: 0.9238041572752317
F1 Score: 0.9010135819212042
Regularized Random Forest Model performance on test set:
Accuracy: 0.9211119459053343
Precision: 0.9123129900137554
Recall: 0.9211119459053343
F1 Score: 0.8976655129642948


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize XGBoost classifier with default settings
xgb_model = XGBClassifier()

# Set hyperparameters for regularization
params = {
    'max_depth': 3,          # Maximum depth of a tree
    'min_child_weight': 1,   # Minimum sum of instance weight (Hessian) needed in a child
    'gamma': 0,              # Minimum loss reduction required to make a further partition
    'reg_alpha': 0.1         # L1 regularization term on weights
}

# Update the model with the regularization parameters
xgb_model.set_params(**params)

# Train the model using your training data
xgb_model.fit(X_train, y_train)

# Make predictions on both training and testing data
y_train_pred_xgb = xgb_model.predict(X_train)
y_test_pred_xgb = xgb_model.predict(X_test)

# Calculate performance metrics for training data
model_train_accuracy_xgb = accuracy_score(y_train, y_train_pred_xgb)
model_train_precision_xgb = precision_score(y_train, y_train_pred_xgb, average='weighted')
model_train_recall_xgb = recall_score(y_train, y_train_pred_xgb, average='weighted')
model_train_f1_xgb = f1_score(y_train, y_train_pred_xgb, average='weighted')

# Calculate performance metrics for test data
model_test_accuracy_xgb = accuracy_score(y_test, y_test_pred_xgb)
model_test_precision_xgb = precision_score(y_test, y_test_pred_xgb, average='weighted')
model_test_recall_xgb = recall_score(y_test, y_test_pred_xgb, average='weighted')
model_test_f1_xgb = f1_score(y_test, y_test_pred_xgb, average='weighted')

# Print performance metrics for training data
print("Regularized XGBoost Model performance on training set:")
print(f"Accuracy: {model_train_accuracy_xgb}")
print(f"Precision: {model_train_precision_xgb}")
print(f"Recall: {model_train_recall_xgb}")
print(f"F1 Score: {model_train_f1_xgb}")
print("===================================")

# Print performance metrics for test data
print("Regularized XGBoost Model performance on test set:")
print(f"Accuracy: {model_test_accuracy_xgb}")
print(f"Precision: {model_test_precision_xgb}")
print(f"Recall: {model_test_recall_xgb}")
print(f"F1 Score: {model_test_f1_xgb}")


Regularized XGBoost Model performance on training set:
Accuracy: 0.9984973703981969
Precision: 0.9984672208584419
Recall: 0.9984973703981969
F1 Score: 0.998479072326577
Regularized XGBoost Model performance on test set:
Accuracy: 0.9923616328575007
Precision: 0.9917383412945238
Recall: 0.9923616328575007
F1 Score: 0.9920243699846734


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
from sklearn.svm import SVC

# Initialize and train the SVM model with regularization
C_value = 1.0  # You can experiment with different values
svm_model_reg = SVC(C=C_value)
svm_model_reg.fit(X_train, y_train)

# Predict on training and test data
y_train_pred_svm_reg = svm_model_reg.predict(X_train)
y_test_pred_svm_reg = svm_model_reg.predict(X_test)

# Evaluate performance on training data
model_train_accuracy_svm_reg, model_train_precision_svm_reg, model_train_recall_svm_reg, model_train_f1_svm_reg = evaluate_model(y_train, y_train_pred_svm_reg)

# Evaluate performance on test data
model_test_accuracy_svm_reg, model_test_precision_svm_reg, model_test_recall_svm_reg, model_test_f1_svm_reg = evaluate_model(y_test, y_test_pred_svm_reg)

# Print performance metrics for training data
print("Regularized SVM Model performance on training set:")
print(f"Accuracy: {model_train_accuracy_svm_reg}")
print(f"Precision: {model_train_precision_svm_reg}")
print(f"Recall: {model_train_recall_svm_reg}")
print(f"F1 Score: {model_train_f1_svm_reg}")
print("===================================")

# Print performance metrics for test data
print("Regularized SVM Model performance on test set:")
print(f"Accuracy: {model_test_accuracy_svm_reg}")
print(f"Precision: {model_test_precision_svm_reg}")
print(f"Recall: {model_test_recall_svm_reg}")
print(f"F1 Score: {model_test_f1_svm_reg}")


Regularized SVM Model performance on training set:
Accuracy: 0.913786626596544
Precision: 0.894423071128919
Recall: 0.913786626596544
F1 Score: 0.8855968268797436
Regularized SVM Model performance on test set:
Accuracy: 0.9144753318307037
Precision: 0.8856306997426185
Recall: 0.9144753318307037
F1 Score: 0.8855813318634765


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 8. Save trained model for deployment

In [44]:
### Create a Pickle file using serialization for Random Forest Classifier 
import pickle
pickle_out = open("RandomForestClassifier.pkl","wb")
pickle.dump(rf_model, pickle_out)
pickle_out.close()

### Create a Pickle file using serialization for XgbootClassifier
import pickle
pickle_out = open("XgbootClassifier.pkl","wb")
pickle.dump(xgb_model, pickle_out)
pickle_out.close()

### Create a Pickle file using serialization for Support Vector Machine
import pickle
pickle_out = open("SVM_reg.pkl","wb")
pickle.dump(svm_model_reg, pickle_out)
pickle_out.close()
